In [1]:
import pandas as pd
import scanpy as sc
import numpy as np
import matplotlib.pyplot as plt
from src.transcript_seg_functions import *


In [3]:

folder_selected1, detected_tanscripts, answer, folder_selected = take_user_input()
d2 = detected_tanscripts[detected_tanscripts['global_z'].isin(answer)]
dict_detected, new_img = get_image(d2) 

3419345


In [4]:

masks = run_cellpose_custom(new_img, folder_selected)




0 1 2 3 4 5 7 8 9 10 11 12 14 15 16 17 18 19 21 22 23 24 25 26 28 29 30 31 32 33 38 39 40 46 

In [5]:
reconstruction, vizualized_reconstruction = create_total_image_from_masks(masks, new_img, folder_selected)

In [6]:
#vizualized_reconstruction = io.imread(folder_selected + os.path.sep +'cellmask_reconstruction.tiff')
  
top3(folder_selected, detected_tanscripts, vizualized_reconstruction)

In [7]:

cell_by_gene, postions_not_in = create_cell_by_gene_(detected_tanscripts, reconstruction, folder_selected)


  0%|          | 0/550 [00:00<?, ?it/s]

In [ ]:
adata, num_transcripts = create_adata_and_add_observations(folder_selected, cell_by_gene, masks, reconstruction)


In [ ]:
plot_qc(adata)
postprocess_adata(adata, num_transcripts, folder_selected)

C:\Users\amonell\Anaconda3\envs\vizgen_cellpose\lib\site-packages\scanpy\preprocessing\_simple.py:373: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)


# For training on the 2 slice set

In [ ]:
import cv2
#create cellpose images for labeling
for i in range(2000, len(new_img[0]), 2000):
    for j in range(2000, len(new_img[1]), 2000):      
        plt.imshow(new_img[i-2000:i, j-2000:j], vmax=1, cmap='gray')
        plt.show()
        value = input("y or n")
        if value == 'y':
            io.imsave(r'C:\Users\amonell\Downloads\label_cellpose_images_allslice\labeling'+str(i)+'_to_'+str(j)+'.tiff', new_img[i-2000:i, j-2000:j])

In [ ]:
import cv2
#used to create images for cellpose training
for i in range(2000, len(new_img[0]), 2000):
    for j in range(2000, len(new_img[1]), 2000):      
        plt.imshow(new_img[i-2000:i, j-2000:j], vmax=1, cmap='gray')
        plt.show()
        value = input("y or n")
        if value == 'y':
            # ksize
            ksize = (5, 5)

            # Using cv2.blur() method 
            image = cv2.blur(new_img[i-2000:i, j-2000:j], ksize) 
            image = image * 255
            io.imsave(r'C:\Users\amonell\Downloads\transcript_images_to_label_slice3\training'+str(i)+'_to_'+str(j)+'.tiff', image)

# For training on all slices

In [18]:
import cv2
#create cellpose images for labeling
for i in range(2000, len(new_img_all[0]), 2000):
    for j in range(2000, len(new_img_all[1]), 2000):      
        plt.imshow(new_img_all[i-2000:i, j-2000:j], vmax=1, cmap='gray')
        plt.show()
        value = input("y or n")
        if value == 'y':
            io.imsave(r'C:\Users\amonell\Downloads\label_cellpose_images_allslice\labeling'+str(i)+'_to_'+str(j)+'.tiff', new_img[i-2000:i, j-2000:j])

In [19]:
import cv2
#used to create images for cellpose training
for i in range(2000, len(new_img_all[0]), 2000):
    for j in range(2000, len(new_img_all[1]), 2000):      
        plt.imshow(new_img_all[i-2000:i, j-2000:j], vmax=1, cmap='gray')
        plt.show()
        value = input("y or n")
        if value == 'y':
            # ksize
            ksize = (5, 5)

            # Using cv2.blur() method 
            image = cv2.blur(new_img_all[i-2000:i, j-2000:j], ksize) 
            image = image * 255
            io.imsave(r'C:\Users\amonell\Downloads\label_cellpose_images_allslice\training'+str(i)+'_to_'+str(j)+'.tiff', image)